# HiRISENet-Tiny

A tiny neural network classifier for Mars HiRISE images

## Introduction

### What is HiRISE?

The Mars HiRISE (High Resolution Imaging Science Experiment) [[1]](#1) is a camera on board the Mars Reconnaissance Orbiter which has been orbiting and studying Mars since 2006. A product of this payload's years of service has been the curation of the Mars orbital image (HiRISE) labeled data set [[2]](#2) by NASA. This dataset provides approximately 73,031 labelled images of the Martian terrain from an orbital perspective. **Figure 1** shows one of the first images taken with the HiRISE camera.

<!--- Public Domain, https://commons.wikimedia.org/w/index.php?curid=656514 >
--->
![First HiRISE Image](../assets/images/mro_first_image.jpg "Figure 1. The first orbital image captured by HiRISE")

**Figure 1. Crop of one of the first images of Mars from the HiRISE camera.**

### The History and Future of AI in Space Exploration

Artificial intelligence, in varying forms, has been a cornerstone of robotic space exploration. In the early days of robotic space exploration, we deployed autonomous rovers and spacecraft that needed to be self-reliant as they navigated uncharted territory. As powerful as our little explorers were, they lacked advanced computing capabilities and still relied on ground-in-the-loop for key decisions. 

Future missions are poised to take greater risk and explore more challenging environments. Because of this, we **need** advanced autonomy. Machine learning has made huge strides in recent times, breaking records and discovering novel things. Because of this rennaissance, the space community, and spacecraft software engineers like me, are very enthusiastic about the possibilities of running deep learning models on future spacecraft.

There are however...some challenges...

1. The black box nature of deep learning models makes it difficult to understand why the model predicts what it predicts. For flight software engineers like me, non-determinism is unsettling to say the least.
2. Deep learning models require **lots** of high-quality data. This is a challenge because space data is either non-existent, very limited, or very bad.
3. Deep learning requires a lot of compute resources. The most advanced computing platform we've but in space as of 2024 is the Ingenuity Mars Helicopter [[3]](#3). This means we need to be really considerate of our model architecture choice.

### Literature Review

The foundational paper this project will be compared to is _Deep Mars: CNN Classification of Mars Imagery for the PDS Imaging Atlas_ by Wagstaff et. al. [[4]](#4). In the paper, the authors set out to train a convolutional neural network to allow scientists to conduct more granular searches on images in NASA's planetary imagery database.
The authors successfully trained an AlexNet model to achieve 99.1%, 88.1%, and 90.6% accuracy across their training, validation, and test sets respectively. A key contributor to their success was taking a pre-trained AleNet model and fine-tuning it to the HiRISE dataset.

### About the Project

#### Problem Statement

Given the resource constraints of space-based computing platforms, can we create an efficient neural network model to perform inference on Mars HiRISE images?

#### Requirements

As is typical in flight software deployments, we need to first establish some requirements:

<<RREQUIREMENTS>>

The challenge I set to myself: Develop a pipeline for training and inference of a low-bit quantized neural network to run on the NVIDIA Jetson Nano 2GB that classifies HiRISE images.

------------------------------

## Setup

In [7]:
import torch
from utils.gpu_management import GPUManager

torch.manual_seed(0xC00FFEE)
torch.use_deterministic_algorithms(True)

device = GPUManager.enable_gpu_if_available()

if device.type == "cuda":
    GPUManager.cleanup_gpu_cache()

__CUDA VERSION: 90100
__Number CUDA Devices: 1
__CUDA Device Name: NVIDIA RTX A2000 8GB Laptop GPU
__CUDA Device Total Memory [GB]: 8.58947584


-------------------------------------

## References

<a id="1">[1]</a> 
Wikipedia contributors. (2024, November 19). HiRISE. Wikipedia. https://en.wikipedia.org/wiki/HiRISE

<a id="2">[2]</a> 
Gary Doran, Emily Dunkel, Steven Lu, & Kiri Wagstaff. (2020). Mars orbital image (HiRISE) labeled data set version 3.2 (3.2.0) [Data set]. Zenodo. https://doi.org/10.5281/zenodo.4002935

<a id="3">[3]</a>
Wikipedia contributors. (2024b, November 19). Ingenuity (helicopter). Wikipedia. https://en.wikipedia.org/wiki/Ingenuity_(helicopter)

<a id="4">[4]</a>
Wagstaff, K., Lu, Y., Stanboli, A., Grimes, K., Gowda, T., & Padams, J. (2018). Deep Mars: CNN Classification of Mars Imagery for the PDS Imaging Atlas. Proceedings of the AAAI Conference on Artificial Intelligence, 32(1). https://doi.org/10.1609/aaai.v32i1.11404